In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import tensorflow as tf
tf.VERSION

'1.15.0'

In [0]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.layers.advanced_activations import LeakyReLU
from functools import partial
from keras.layers.merge import _Merge
import keras.backend as K
import numpy as np

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
# Following parameter and optimizer set as recommended in paper
n_critic = 5
optimizer = RMSprop(lr=0.00005)

class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])
        
def Generator():
  model = Sequential()
  model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
  model.add(Reshape((7, 7, 128)))
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size=4, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(UpSampling2D())
  model.add(Conv2D(64, kernel_size=4, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Activation("relu"))
  model.add(Conv2D(channels, kernel_size=4, padding="same"))
  model.add(Activation("tanh"))
  model.summary()

  noise = Input(shape=(latent_dim,))
  img = model(noise)

  return Model(inputs = noise, outputs = img)

def Critic():
  model = Sequential()
  model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
  model.add(ZeroPadding2D(padding=((0,1),(0,1))))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
  model.add(BatchNormalization(momentum=0.8))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(1))
  model.summary()

  img = Input(shape = img_shape)
  validity = model(img)
        
  return Model(inputs = img, outputs = validity)

def wasserstein_loss(y_true, y_pred):
  return K.mean(y_true * y_pred)

def gradient_penalty_loss(y_true, y_pred, averaged_samples):
  """
  Computes gradient penalty based on prediction and weighted real / fake samples
  """
  gradients = K.gradients(y_pred, averaged_samples)[0]
  # compute the euclidean norm by squaring ...
  gradients_sqr = K.square(gradients)
  #   ... summing over the rows ...
  gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
  #   ... and sqrt
  gradient_l2_norm = K.sqrt(gradients_sqr_sum)
  # compute lambda * (1 - ||grad||)^2 still for each single sample
  gradient_penalty = K.square(1 - gradient_l2_norm)
  # return the mean as loss over all the batch samples
  return K.mean(gradient_penalty)        


# Build and compile the critic
C = Critic()
C.compile(loss = wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])

# Build the generator
G = Generator()

# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = G(z)

# For the combined model we will only train the generator
C.trainable = False

# The discriminator takes generated images as input and determines validity
valid = C(img)

# The combined model  (stacked generator and critic)
# Trains the generator to fool the discriminator
combined = Model(inputs = z, outputs = valid)
combined.compile(loss = wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])




# Build the generator and critic
G = Generator()
C = Critic()

#-------------------------------
# Construct Computational Graph
#       for the Critic
#-------------------------------

# Freeze generator's layers while training critic
G.trainable = False

# Image input (real sample)
real_img = Input(shape=img_shape)

# Noise input
z_disc = Input(shape=(latent_dim,))
# Generate image based of noise (fake sample)
fake_img = G(z_disc)

# Discriminator determines validity of the real and fake images
fake = C(fake_img)
valid = C(real_img)

# Construct weighted average between real and fake images
interpolated_img = RandomWeightedAverage()([real_img, fake_img])
# Determine validity of weighted sample
validity_interpolated = C(interpolated_img)

# Use Python partial to provide loss function with additional
# 'averaged_samples' argument
partial_gp_loss = partial(gradient_penalty_loss, averaged_samples=interpolated_img)
partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

C_model = Model(inputs=[real_img, z_disc], outputs=[valid, fake, validity_interpolated])
C_model.compile(loss=[wasserstein_loss, wasserstein_loss, partial_gp_loss], optimizer=optimizer, loss_weights=[1, 1, 10])

#-------------------------------
# Construct Computational Graph for Generator
#-------------------------------

# For the generator we freeze the critic's layers
C.trainable = False
G.trainable = True

# Sampled noise for input to generator
z_gen = Input(shape=(latent_dim,))
# Generate images based of noise
img = G(z_gen)
# Discriminator determines validity
valid = C(img)
# Defines generator model
G_model = Model(z_gen, valid)
G_model.compile(loss=wasserstein_loss, optimizer=optimizer)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 8, 8, 32)         

In [0]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import os
import numpy as np

os.chdir('/content/drive/My Drive/GitHub Repositories/Generative Models')
baseDir = './Improved Training of Wasserstein GANs'
outputDir = os.path.join(baseDir, 'outputs')
if not os.path.exists(outputDir):
  os.makedirs(outputDir)
  print('Output Directory Created to save Results')

epochs=20000
batch_size=32
sample_interval=50

# Load the dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

# Adversarial ground truths
valid = -np.ones((batch_size, 1))
fake =  np.ones((batch_size, 1))
dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig(outputDir+'/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()
    plt.close()


for epoch in range(epochs):
  epoch+=1
  for _ in range(n_critic):
    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    # Sample generator input
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    # Train the critic
    d_loss = C_model.train_on_batch([imgs, noise], [valid, fake, dummy])

  # ---------------------
  #  Train Generator
  # ---------------------

  g_loss = G_model.train_on_batch(noise, valid)

  # Plot the progress
  print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss[0], g_loss))

  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
      save_imgs(epoch)






In [0]:
import imageio
import glob
images = []
for file_name in os.listdir(outputDir):
    if file_name.endswith('.png'):
        file_path = os.path.join(outputDir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave(os.path.join(baseDir, 'WGANGP-training-outputs.gif'), images, fps = 3)